In [87]:
%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

import nltk
from nltk.corpus import stopwords
from os.path import join
from glob import glob

from time import time
from scipy.sparse import csr_matrix, lil_matrix

env: JOBLIB_TEMP_FOLDER=/tmp


In [102]:
with open('./texts_test', 'rb') as f:
    texts_test = pickle.load(f)

count_vect = joblib.load('./models/countVect.pkl')
dataset = count_vect.transform(texts_test)

lda = joblib.load('./models/lda.pkl')

In [103]:
class TopicModeler(object):
    '''
    Inteface object for CountVectorizer + LDA simple
    usage.
    '''
    def __init__(self, count_vect, lda):
        '''
        Args:
             count_vect - CountVectorizer object from sklearn.
             lda - LDA object from sklearn.
        '''
        self.lda = lda
        self.count_vect = count_vect
        self.count_vect.input = 'content'
        
    def __call__(self, text):
        '''
        Gives topics distribution for a given text
        Args:
             text - raw text via python string.
        returns: numpy array - topics distribution for a given text.
        '''
        vectorized = self.count_vect.transform([text])
        lda_topics = self.lda.transform(vectorized)
        return lda_topics
    def get_keywords(self, text, n_topics=3, n_keywords=5):
        '''
        For a given text gives n top keywords for each of m top texts topics.
        Args:
             text - raw text via python string.
             n_topics - int how many top topics to use.
             n_keywords - how many top words of each topic to return.
        returns:
                list - of m*n keywords for a given text.
        '''
        lda_topics = self(text)
        lda_topics = np.squeeze(lda_topics, axis=0)
        n_topics_indices = lda_topics.argsort()[-n_topics:][::-1]
        
        top_topics_words_dists = []
        for i in n_topics_indices:
            top_topics_words_dists.append(self.lda.components_[i])
        
        keywords = np.zeros(shape=(n_keywords*n_topics, self.lda.components_.shape[1]))
        for i,topic in enumerate(top_topics_words_dists):
            n_keywords_indices = topic.argsort()[-n_keywords:][::-1]
            for k,j in enumerate(n_keywords_indices):
                keywords[i * n_keywords + k, j] = 1
        keywords = self.count_vect.inverse_transform(keywords)
        keywords = [keyword[0] for keyword in keywords]
        return keywords  

In [104]:
print(texts_test[0])

Текст расследования должны опубликовать 18 апреля. Юристы президента США Дональда Трампа договорились создать стратегию для опровержения любой вредной информации из расследования Мюллера. Об этом сообщает New York Time со ссылкой на источники, близкие к администрации президента. Как сообщает издание, юристы президента решили разделить текст доклада спецпрокурора Мюллера между собой, чтобы ознакомиться с его содержанием и выпустить длинный ответ, который опровергнет любую потенциально вредную информацию. Как сообщает DW, Минюст США заявил, что Доклад спецпрокурора Мюллера опубликуют 18 апреля. Отредактированная версия доклада также будет направлена в Конгресс США. По словам генерального прокурора США Уильяма Барра, часть данных будет недоступна для общественности. Он заявил, что подобные меры коснутся информации об источниках спецслужб, а также данных, затрагивающих частную жизнь «второстепенных действующих лиц», которым не предъявлены какие-либо обвинения. 22 марта завершилось расследо

In [105]:
tm = TopicModeler(count_vect, lda) 

In [106]:
key_words = tm.get_keywords(texts_test[1], n_topics=10, n_keywords=5)
print(key_words)

['полиция', 'мужчина', 'летний', 'женщина', 'житель', 'автомобиль', 'tesla', 'маск', 'час', 'маска', 'covid', 'covid19', 'коронавирус', 'вирус', 'здоровье', 'сми', 'издание', 'медиа', 'источник', 'рбк', 'москва', 'санкт', 'петербург', 'мэр', 'губернатор', 'канал', 'телеграм', 'животное', 'кот', 'кошка', 'гарри', 'джон', 'life', 'www', 'рекорд', 'суд', 'дело', 'рф', 'ук', 'решение', 'трамп', 'times', 'new', 'york', 'президент', 'фильм', 'кино', 'роль', 'картина', 'герой']


In [93]:

sorted_words_coeffs = lda.components_.argsort(axis=1)
n_top = 10
top_coefs = sorted_words_coeffs[:,-n_top:][:,::-1]

#making those texts consisting of top words
vect_texts = np.zeros((top_coefs.shape[0], lda.components_.shape[1]),
                       )
for i,n_top_coefs in enumerate(top_coefs):
    for coef in n_top_coefs:
        vect_texts[i,coef] = 1


In [94]:

top_words = count_vect.inverse_transform(vect_texts)
top_words_set = set()
for words in top_words:
    top_words_set.update(set(words))
print(len(top_words_set))


544


In [95]:

voc_to_idf = {word : i for i, word in enumerate(top_words_set)}

with open('./stopwords/stopwords_updated.pkl', 'rb') as f:
    stopwords = pickle.load(f)

with open('./texts_train', 'rb') as f:
    texts_train = pickle.load(f)


In [96]:

tfidf_tw = TfidfVectorizer(input='content', vocabulary=voc_to_idf, stop_words=stopwords)
tfidf_tw.fit(texts_train)

idfs = tfidf_tw.idf_
print(idfs.shape)


(544,)


In [97]:

#computing n most common words
n_top = int(idfs.shape[0] * 0.2)
n_top_indices = np.argsort(idfs)[:n_top]

vect_words = np.zeros((n_top, len(idfs)))


In [98]:

inv_voc_to_idf = { voc_to_idf[key] : key for key in voc_to_idf.keys()}
extra_stop_words = []
for ind in n_top_indices:
    extra_stop_words.append(inv_voc_to_idf[ind])
print(len(extra_stop_words))


108


In [99]:
print(extra_stop_words)

['кроме', 'сми', 'дело', 'место', 'дома', 'решение', 'деньги', 'вопрос', 'жизнь', 'назад', 'глава', 'пресс', 'рф', 'работать', 'суд', 'использовать', 'данные', 'фото', 'интернет', 'издание', 'проект', 'президент', 'история', 'друг', 'политика', 'месяц', 'россия', 'интервью', 'пост', 'youtube', 'мир', 'автор', 'главный', 'google', 'работа', 'полиция', 'вконтакте', 'правда', 'сайт', 'процесс', 'число', 'онлайн', 'доступ', 'город', 'сложно', 'мужчина', 'система', 'путин', 'источник', 'apple', 'дом', 'сервис', 'фильм', 'бизнес', 'канал', 'директор', 'проблема', 'группа', 'уровень', 'право', 'летний', 'представитель', 'говорить', 'закон', 'кино', 'срок', 'роль', 'таки', 'девушка', 'медиа', 'правительство', 'ролик', 'шоу', 'стоимость', 'важно', 'название', 'пользователь', 'центр', 'команда', 'кстати', 'бывший', 'интересно', 'домой', 'идея', 'купить', 'министр', 'мвд', 'женщина', 'алексей', 'telegram', 'смотреть', 'путь', 'приложение', 'ночь', 'опыт', 'covid', 'расследование', 'вид', 'вариант

In [100]:
extra_stop_words = ['кроме', 'назад', 'говорить', 'обязательно']

In [101]:

stopwords = stopwords + extra_stop_words
#serializing all-merged stopwords set
with open('./stopwords/stopwords_updated.pkl', 'wb') as f:
    pickle.dump(stopwords, f)
